In [ ]:
"""
Format: term_i, df_i:
    <doc1, tf_1: <pos1, pos2, … >; 
    doc2, tf_2: <pos1, pos2, …>; …>

Format: to, 993427: 
    <1, 6: <7, 18, 33, 72, 86, 231>;   
    2, 5: <1, 17, 74, 222, 255>; … >
"""

In [30]:
import re
import math

In [12]:
inverted_index = eval(open('all_docs_freq_all_tokens_dict.txt','r').read())

In [14]:
inverted_index['usernam, 2']

[{'13185,1': [0]}, {'13191,8': [841, 848, 850, 948, 955, 969, 980, 982]}]

In [55]:
# ref: https://mathpretty.com/10661.html
term = "usernam"

NUM_OF_ALL_DOCS = 13173 # number of all documents (processed)

def calculate_weight(raw_count_in_doc, term_doc_in_all_document):
    """
    raw_count_in_doc : appear times of term in the doc (raw count)
    term_doc_in_all_document : number of docs which contains the term (processed)
    """
    idf = NUM_OF_ALL_DOCS / term_doc_in_all_document
    w_i_j = (1 + math.log10(raw_count_in_doc)) * math.log10(idf)

    # print((1 + math.log10(raw_count_in_doc)), math.log10(idf))

    return w_i_j

calculate_weight(1, 2)

1.0 3.818654696160069


3.818654696160069

In [51]:
term_doc_in_all_document = 0

keys = ['usernam, 2', 'member, 3']

d_j = []
q = []

# for term_df_key in list(inverted_index.keys()):
for term_df_key in list(keys):
    d_j = []
    term = re.findall('\w+', term_df_key)[0] # 'usernam, 2' -> 'usernam'
    term_doc_in_all_document = int(re.findall(', ([0-9]+)', term_df_key)[0]) # 'usernam, 2' -> 2

    for i in range(len(inverted_index[term_df_key])):
        doc_raw_count_and_posting = list(inverted_index[term_df_key][i].keys())[0] # [{'13185,1': [0]}, {'13191,8': ...}...]
        doc_id = int(re.findall('([0-9]+),', doc_raw_count_and_posting)[0]) # '13185,1' -> 13185
        raw_count_in_doc = int(re.findall(',([0-9]+)', doc_raw_count_and_posting)[0]) # '13185,1' -> 1

        weight_i_j = calculate_weight(raw_count_in_doc, term_doc_in_all_document)
        d_j.append(weight_i_j)
        # print(doc_id, term, weight_i_j, raw_count_in_doc)
    print(term, doc_id, d_j)

# d_j

usernam 13191 [8.792777378669882, 27.076843926618768]
member 13193 [30.52191019395961, 8.387312270561717, 8.387312270561717]


In [56]:
# Vectors
vec_a = [1, 2, 3, 4, 5]
vec_b = [1, 3, 5, 7, 9]

dot = sum(i * j for i, j in zip(vec_a, vec_b))
a_norm = sum(i**2 for i in vec_a) ** 0.5
b_norm = sum(i**2 for i in vec_b) ** 0.5

cos_sim = dot / (a_norm * b_norm)
print(cos_sim)

0.9972413740548081


In [ ]:
# # cosine similarity
# term = "usernam"
# query = "username"

# query -> stemmize -> search key (use cosine similarity to get most similiar key) -> get weight

# d_j = query 在 doc 的 tf-idf weight 向量
# q = query 各字在 query 的 tf-idf weight 向量
Hong Kong Business
1 1 1
Hong Kong Kong Business